In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
!source /content/drive/MyDrive/colab_env/bin/activate

In [ ]:
! pip install pyet swifter

In [48]:
import pandas as pd
import pyarrow
import pyet
import math
import datetime
import swifter
import numpy as np

In [49]:
prism_data_daily = pd.read_parquet('/content/drive/MyDrive/EC_Tower/merged_prism_data_par_for_ameri_stations.parquet', engine='pyarrow')


In [50]:
# convert lat to radian
prism_data_daily["latitude_in_radian"] = list(map(math.radians, prism_data_daily["latitude"]))

prism_data_daily['time'] = pd.to_datetime(prism_data_daily['date'],format= "%Y-%m-%d" )

prism_data_daily.set_index('time', inplace=True)
prism_data_daily

,system:index,date,point_id,ppt,.geo,tdmean,tmean,vpdmax,vpdmin,tmin,tmax,latitude,longitude,Site_ID,General_classification,Elevation,Energy_balance,Land_cover_details,Land_cover_type,latitude_in_radian
time,,,,,,,,,,,,,,,,,,,,
2008-12-31,0_20090101,2008-12-31,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.717,5.503000,0.912,0.239,4.565,6.442,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
2009-01-01,0_20090102,2009-01-01,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.025,5.944000,1.700,0.494,4.836,7.053,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
2009-01-02,0_20090103,2009-01-02,1,2.898,"{""type"":""MultiPoint"",""coordinates"":[]}",4.454,6.111000,2.067,0.370,4.656,7.567,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
2009-01-03,0_20090104,2009-01-03,1,0.228,"{""type"":""MultiPoint"",""coordinates"":[]}",3.300,5.052000,3.851,0.202,0.441,9.665,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
2009-01-04,0_20090105,2009-01-04,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",1.732,4.622000,3.458,0.290,0.316,8.929,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-26,12_20201227,2020-12-26,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",6.645,11.471399,8.840,0.435,6.817,16.126,38.2006,-122.0264,US-Srr,Wetland/Riparian,8.0,0.86,Brackish tidal marsh,Wetlands,0.666726
2020-12-27,12_20201228,2020-12-27,13,1.227,"{""type"":""MultiPoint"",""coordinates"":[]}",4.315,9.327499,7.957,0.464,4.567,14.088,38.2006,-122.0264,US-Srr,Wetland/Riparian,8.0,0.86,Brackish tidal marsh,Wetlands,0.666726
2020-12-28,12_20201229,2020-12-28,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.685,9.029900,6.475,0.323,4.429,13.631,38.2006,-122.0264,US-Srr,Wetland/Riparian,8.0,0.86,Brackish tidal marsh,Wetlands,0.666726


In [51]:
# Solar Constant [ MJ m-2 min-1]
SOLAR_CONSTANT = 0.0820

def standard_date_to_Julian_day (
    standard_date_in_gregorian : str
    ) -> int:

    """
    Description
    -----------
    calculate Julian Day with standard date
    Ref:https://rafatieppo.github.io/post/2018_12_01_juliandate/
    ----------

    standard_date_in_gregorian : str
        Date with the specified standard

    Returns
    -------
    Julian Day : int
        Number of days of the year taking into account the leap year
    """

    fmt='%Y-%m-%d'
    sdtdate = datetime.datetime.strptime(standard_date_in_gregorian, fmt)
    sdtdate = sdtdate.timetuple()
    jdate = sdtdate.tm_yday

    return(jdate)


def inverse_relative_distance_earth_sun(
    julian_date: int
) -> float:

    """
    Description
    -----------
    calculate Inverse Relative Distance Earth Sun With Julian Date - eq 23 FAO56
    ----------


    Julian Day : int
        Number of days of the year taking into account the leap year

    Returns
    -------
    inverse_relative_distance_earth_sun : float
        inverse relative distance earth sun in Radian
    """

    return 1 + (0.033 * np.cos((2 * np.pi * julian_date)/365))

def solar_declination(
    julian_date: int
) -> float:

    """
    Description
    -----------
    calculate Solar Declination With Julian Date - eq 24 FAO56
    ----------

    julian_date : int
        Number of days of the year taking into account the leap year

    Returns
    -------
    solar_declination : float
        solar_declination in Radian
    """

    return 0.409 * np.sin(((2 * np.pi * julian_date) / 365) - 1.39)

def sunset_hour_angle(
    latitude : float,
    solar_declination : float
) -> float:

    """
    Description
    -----------
    calculate Sunset Hour Angle With Latitude and Solar Declination - eq 25 FAO56
    ----------

    latitude: float
        latitude in Radian
    solar_declination : float
        solar_declination in Radian

    Returns
    -------
    sunset_hour_angle : float
        sunset hour angle in Radian
    """

    return np.arccos(-np.tan(latitude) * np.tan(solar_declination))

def extraterrestrial_radiation(
    inverse_relative_distance_earth_sun : float,
    sunset_hour_angle : float,
    latitude : float,
    solar_declination : float
) -> float:

    """
    Description
    -----------
    calculate Extraterrestrial Radiation - eq 28 FAO56
    ----------

    inverse_relative_distance_earth_sun : float
        inverse relative distance earth sun in Radian
    sunset_hour_angle : float
        sunset hour angle in Radian
    latitude: float
        latitude in Radian
    solar_declination : float
        solar_declination in Radian

    Returns
    -------
    extraterrestrial_radiation : float
            extraterrestrial radiation in MJ/m**2/day
    """

    temp_1 = ((24 * 60 )/ np.pi) * SOLAR_CONSTANT * inverse_relative_distance_earth_sun

    temp_2 = sunset_hour_angle * np.sin(latitude) * np.sin(solar_declination)

    temp_3 = np.cos(latitude)* np.cos(solar_declination) * np.sin(sunset_hour_angle)

    return temp_1 * (temp_2 + temp_3)

def solar_or_Short_wave_Radiation_Hargreaves(
    extraterrestrial_radiation : float,
    tmax : float,
    tmin : float,
    Adjustment_coefficient_or_K_RS : float
)-> float:

    """
    Description
    -----------
    calculate Solar or shortwave radiation with max and min Tempreture - eq 50 FAO56
    ----------

    extraterrestrial_radiation : float
        extraterrestrial radiation in MJ/m**2/day
    tmax : float
        Maximum Temperature in celsius
    tmin : float
        Minimum Temperature in celsius
    Adjustment_coefficient_or_K_RS : float
        Adjustment coefficient in C**-0.5 -- between 0.16 to 0.19
            for interior locations, where land mass dominates and air masses are not strongly
            influenced by a large water body, kRs ≅ 0.16;
            for coastal locations, situated on or adjacent to the coast of a large land mass and where
            air masses are influenced by a nearby water body, kRs ≅ 0.19

    Returns
    -------
    Solar or shortwave radiation : float
        Solar or shortwave radiation in MJ/m**2/day
    """
    temp_1 = extraterrestrial_radiation


    return Adjustment_coefficient_or_K_RS * np.sqrt(tmax - tmin) * temp_1

In [52]:
Adjustment_coefficient_or_K_RS = 0.175

array_Date = prism_data_daily.date.to_numpy()

array_latitude = prism_data_daily.latitude_in_radian.to_numpy()

array_tmin = prism_data_daily["tmin"].to_numpy()

array_tmax = prism_data_daily["tmax"].to_numpy()


julian_day = np.vectorize(standard_date_to_Julian_day)(array_Date)
dr = np.vectorize(inverse_relative_distance_earth_sun)(julian_day)

delta = np.vectorize(solar_declination)(julian_day)

omega_s = np.vectorize(sunset_hour_angle)(array_latitude, delta)

ra = np.vectorize(extraterrestrial_radiation)(dr, omega_s, array_latitude, delta)

rs = np.vectorize(solar_or_Short_wave_Radiation_Hargreaves)(ra, array_tmax, array_tmin, Adjustment_coefficient_or_K_RS)


# Append each array as a new column
prism_data_daily['julian_day'] = julian_day
prism_data_daily['dr'] = dr
prism_data_daily['delta'] = delta
prism_data_daily['omega_s'] = omega_s
prism_data_daily['ra'] = ra
prism_data_daily['rs'] = rs
prism_data_daily

,system:index,date,point_id,ppt,.geo,tdmean,tmean,vpdmax,vpdmin,tmin,...,Energy_balance,Land_cover_details,Land_cover_type,latitude_in_radian,julian_day,dr,delta,omega_s,ra,rs
time,,,,,,,,,,,,,,,,,,,,,
2008-12-31,0_20090101,2008-12-31,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.717,5.503000,0.912,0.239,4.565,...,1.00,Peach,Orchards,0.636324,366,1.032995,-0.401008,1.252175,16.025685,3.842260
2009-01-01,0_20090102,2009-01-01,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.025,5.944000,1.700,0.494,4.836,...,1.00,Peach,Orchards,0.636324,1,1.032995,-0.401008,1.252175,16.025685,4.175779
2009-01-02,0_20090103,2009-01-02,1,2.898,"{""type"":""MultiPoint"",""coordinates"":[]}",4.454,6.111000,2.067,0.370,4.656,...,1.00,Peach,Orchards,0.636324,2,1.032980,-0.399564,1.253500,16.080602,4.801329
2009-01-03,0_20090104,2009-01-03,1,0.228,"{""type"":""MultiPoint"",""coordinates"":[]}",3.300,5.052000,3.851,0.202,0.441,...,1.00,Peach,Orchards,0.636324,3,1.032956,-0.398001,1.254930,16.139901,8.578247
2009-01-04,0_20090105,2009-01-04,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",1.732,4.622000,3.458,0.290,0.316,...,1.00,Peach,Orchards,0.636324,4,1.032922,-0.396320,1.256466,16.203566,8.321965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-26,12_20201227,2020-12-26,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",6.645,11.471399,8.840,0.435,6.817,...,0.86,Brackish tidal marsh,Wetlands,0.666726,361,1.032922,-0.406440,1.225258,14.737636,7.868961
2020-12-27,12_20201228,2020-12-27,13,1.227,"{""type"":""MultiPoint"",""coordinates"":[]}",4.315,9.327499,7.957,0.464,4.567,...,0.86,Brackish tidal marsh,Wetlands,0.666726,362,1.032956,-0.405594,1.226097,14.770615,7.975867
2020-12-28,12_20201229,2020-12-28,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.685,9.029900,6.475,0.323,4.429,...,0.86,Brackish tidal marsh,Wetlands,0.666726,363,1.032980,-0.404627,1.227054,14.808078,7.861001


In [53]:
# prism_data_daily["blaney_criddle"] = pyet.blaney_criddle(
#     tmean = prism_data_daily["tmean"],
#     lat = prism_data_daily["latitude_in_radian"]
# )

# prism_data_daily["hamon"] = pyet.hamon(
#     tmean = prism_data_daily["tmean"],
#     lat = prism_data_daily["latitude_in_radian"]
# )

prism_data_daily["oudin"] = pyet.oudin(
    tmean = prism_data_daily["tmean"],
    lat = prism_data_daily["latitude_in_radian"]
)

prism_data_daily["hargreaves"] = pyet.hargreaves(
    tmean = prism_data_daily["tmean"],
    tmax = prism_data_daily["tmax"],
    tmin = prism_data_daily["tmin"],
    lat = prism_data_daily["latitude_in_radian"]
)

prism_data_daily["abtew"] = pyet.abtew(
    tmean = prism_data_daily["tmean"],
    rs = prism_data_daily["rs"]
)


prism_data_daily["mcguinness_bordne"] = pyet.mcguinness_bordne(
    tmean = prism_data_daily["tmean"],
    lat = prism_data_daily["latitude_in_radian"]
)

# prism_data_daily["jensen_haise"] = pyet.jensen_haise(
#     tmean = prism_data_daily["tmean"],
#     lat = prism_data_daily["latitude_in_radian"]
# )
prism_data_daily = prism_data_daily.reset_index()

In [58]:
prism_data_daily.columns.values.tolist()

['time',
 'system:index',
 'date',
 'point_id',
 'ppt',
 '.geo',
 'tdmean',
 'tmean',
 'vpdmax',
 'vpdmin',
 'tmin',
 'tmax',
 'latitude',
 'longitude',
 'Site_ID',
 'General_classification',
 'Elevation',
 'Energy_balance',
 'Land_cover_details',
 'Land_cover_type',
 'latitude_in_radian',
 'julian_day',
 'dr',
 'delta',
 'omega_s',
 'ra',
 'rs',
 'oudin',
 'hargreaves',
 'abtew',
 'mcguinness_bordne',
 'month',
 'year']

In [60]:
#  Create a new column for the month


prism_data_daily["month"] = prism_data_daily['time'].dt.month
prism_data_daily["year"] = prism_data_daily['time'].dt.year


# 3. Group by month and site_id, then aggregate the data
monthly_data = prism_data_daily.groupby(['Site_ID', 'month']).agg({
    'oudin' : 'sum',
    'hargreaves' : 'sum',
    'abtew' : 'sum',
    'mcguinness_bordne' : 'sum',
}).reset_index()

monthly_data


,Site_ID,month,oudin,hargreaves,abtew,mcguinness_bordne
0,Almond_High,1,375.020823,548.225800,819.061030,551.280609
1,Almond_High,2,493.201590,733.431161,1023.386044,725.006338
2,Almond_High,3,824.282827,1192.996944,1515.707979,1211.695756
3,Almond_High,4,1139.804574,1641.487014,1897.620580,1675.512724
4,Almond_High,5,1530.637136,2145.317166,2258.402792,2250.036589
...,...,...,...,...,...,...
151,US-Twt,8,1547.226640,2057.531040,2037.938899,2274.423160
152,US-Twt,9,1199.531034,1614.082477,1633.649185,1763.310620
153,US-Twt,10,802.784104,1109.185993,1248.189646,1180.092633
154,US-Twt,11,430.600646,619.509636,836.241525,632.982950
